In [1]:
import hickle as hkl

In [ ]:
#这一个是负样本均衡过的
import hickle as hkl
x_val = hkl.load('./Gen_data/13_x_val.hkl')
y_val = hkl.load('./Gen_data/13_y_val.hkl')
x_train = hkl.load('./Gen_data/13_x_train.hkl')
y_train = hkl.load('./Gen_data/13_y_train.hkl')

# Make Network

In [3]:
import numpy as np
import pandas as pd
import sys
import os

os.environ['KERAS_BACKEND']='theano'
from keras.engine.topology import Layer
from keras import initializations
from keras import backend as K

from collections  import defaultdict
import pandas as pd
import numpy as np

class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializations.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):


        M = K.tanh(x)
        alpha = K.dot(M,self.W)#.dimshuffle(0,2,1)

        ai = K.exp(alpha)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return K.tanh(weighted_input.sum(axis=1))
        '''
        eij = K.tanh(K.dot(x, self.W))

        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')

        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)
        '''
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])


Using Theano backend.
WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: CUDA driver version is insufficient for CUDA runtime version)


In [4]:
from keras.layers import Embedding
from keras.layers import Dense,Input,Activation
from keras.layers import Embedding, LSTM, Bidirectional,GRU,InputLayer
from keras.models import Model,Sequential
from  keras.regularizers import ActivityRegularizer
from keras.layers.core import Dropout,Flatten

In [ ]:
import keras
keras.__version__

In [5]:
from keras import layers

In [15]:
from keras.layers.merge import oncatenate

ImportError: No module named merge

In [6]:
kmer_input = Input(shape=(4096,2), dtype='float32')

In [7]:
l_lstm =Bidirectional(LSTM(5,return_sequences=True))(kmer_input)

In [8]:
l_lstm_drop = Dropout(0.3)(l_lstm)
l_att = AttLayer()(l_lstm_drop)
#l_flatten = Flatten()(l_att)
l_att_drop = Dropout(0.5)(l_att)
l_dense = Dense(128)(l_att_drop)
l_dense_1 = Dense(32)(l_dense)
preds = Dense(len( y_train[0]), activation='softmax',activity_regularizer= ActivityRegularizer(l2=0.005))(l_dense_1)
model  = Model (kmer_input,preds)

In [9]:
model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - attention LSTM network")
model.summary()


model fitting - attention LSTM network
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 4096, 2)       0                                            
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 4096, 10)      320         input_1[0][0]                    
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 4096, 10)      0           bidirectional_1[0][0]            
____________________________________________________________________________________________________
attlayer_1 (AttLayer)            (None, 10)            10          dropout_1[0][0]                  
____________________________________________________

In [10]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=20, batch_size=100)

INFO (theano.gof.compilelock): Waiting for existing lock by process '7393' (I am process '11960')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/yinqijin/.theano/compiledir_Linux-3.10-el7.x86_64-x86_64-with-centos-7.2.1511-Core-x86_64-2.7.12-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '7393' (I am process '11960')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/yinqijin/.theano/compiledir_Linux-3.10-el7.x86_64-x86_64-with-centos-7.2.1511-Core-x86_64-2.7.12-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '7393' (I am process '11960')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/yinqijin/.theano/compiledir_Linux-3.10-el7.x86_64-x86_64-with-centos-7.2.1511-Core-x86_64-2.7.12-64/lock_dir
INFO (theano.gof.compilelock): Waiting for existing lock by process '7393' (I am process '11960')
INFO (theano.gof.compilelock): To manually release the lock

Train on 3376 samples, validate on 844 samples
Epoch 1/20
3376/3376 [==============================] - 181s - loss: 0.5005 - acc: 0.5258 - val_loss: 0.4887 - val_acc: 0.6114
Epoch 2/20
3376/3376 [==============================] - 184s - loss: 0.4964 - acc: 0.5584 - val_loss: 0.4932 - val_acc: 0.5249
Epoch 3/20
3376/3376 [==============================] - 181s - loss: 0.4967 - acc: 0.5554 - val_loss: 0.4883 - val_acc: 0.5640
Epoch 4/20
3376/3376 [==============================] - 176s - loss: 0.4950 - acc: 0.5616 - val_loss: 0.4860 - val_acc: 0.6102
Epoch 5/20
3376/3376 [==============================] - 130s - loss: 0.4952 - acc: 0.5628 - val_loss: 0.4862 - val_acc: 0.5995
Epoch 6/20
3376/3376 [==============================] - 110s - loss: 0.4946 - acc: 0.5746 - val_loss: 0.4871 - val_acc: 0.5652
Epoch 7/20
3376/3376 [==============================] - 111s - loss: 0.4940 - acc: 0.5755 - val_loss: 0.4871 - val_acc: 0.5806
Epoch 8/20
3376/3376 [==============================] - 110s - l

In [ ]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=70, batch_size=100)

In [ ]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=20, batch_size=100)

In [ ]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=20, batch_size=100)

In [ ]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=50, batch_size=100)

In [ ]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=50, batch_size=100)

In [ ]:
pro = model.predict_on_batch(x_val)

In [ ]:
print pro

# Save&Search

In [ ]:
dir(model)

In [ ]:
model.__dict__

In [ ]:
model.save_weights('./Gen_data/10_Att-BLSTM.h5')

In [ ]:
model.load_weights('./Gen_data/10_Att-BLSTM.h5',by_name=True)